<a href="https://colab.research.google.com/github/ProjectProRepo/How-to-Build-an-LLM-Powered-Data-Analysis-Agent-/blob/main/LLM_Powered_Data_Analysis_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -qqq install pip --progress-bar off
!pip -qqq install 'crewai[tools]'==0.28.8 --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install duckduckgo-search==5.3.0 --progress-bar off
!pip install exa-py

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but yo

In [ ]:
from exa_py import Exa
import requests
import pandas as pd
import os

os.environ["GROQ_API_KEY"] = input("Enter your GROQ API Key: ")

In [4]:
# Initialize Exa API
exa = Exa(api_key="Your EXA API Key")

def get_cryptocurrency_news(query):
    result = exa.search_and_contents(query, summary=True)

    # Ensure results exist before iterating
    if result.results:
        news_list = []
        for item in result.results:
            # Extracting attributes directly from the Result object
            news_item = {
                "title": item.title if hasattr(item, "title") else "No Title",
                "url": item.url if hasattr(item, "url") else "#",  # URL and ID are the same
                "id": item.id if hasattr(item, "id") else "#",  # ID is same as URL
                "score": item.score if hasattr(item, "score") else "No Score",
                "published_date": item.published_date if hasattr(item, "published_date") else "Unknown Date",
                "author": item.author if hasattr(item, "author") else "Unknown Author",
                "image": item.image if hasattr(item, "image") else "No Image",
                "favicon": item.favicon if hasattr(item, "favicon") else "No Favicon",
                "summary": item.summary if hasattr(item, "summary") else "No Summary",
                "highlights": item.highlights if hasattr(item, "highlights") else "No Highlights",
                "highlight_scores": item.highlight_scores if hasattr(item, "highlight_scores") else "No Highlight Scores",
            }
            news_list.append(news_item)

        return news_list
    else:
        return []  # Return empty list if no results found


# Fetch and display news
news = get_cryptocurrency_news("Bitcoin cryptocurrency")
for article in news[:5]:  # Display top 5 articles
    print(f"Title: {article['title']}\nURL: {article['url']}\nID: {article['id']}\nPublished Date: {article['published_date']}\nAuthor: {article['author']}\nImage: {article['image']}\nFavicon: {article['favicon']}\nSummary: {article['summary']}\n")



Title: What is Bitcoin? Introductory video and current Bitcoin price.
URL: https://www.weusecoins.com/
ID: https://www.weusecoins.com/
Published Date: 2022-01-01T00:00:00.000Z
Author: WeUseCoins
Image: None
Favicon: https://www.weusecoins.com/favicon.ico
Summary: Bitcoin is a decentralized, peer-to-peer payment network, the first practical implementation of blockchain technology.  It allows secure, easy, and relatively low-fee transactions globally.  While its price is volatile, Bitcoin is considered by some notable figures like Al Gore and Bill Gates to be a significant technological advancement.  It's open-source, with no single owner or central authority.  The network is maintained by a global community of developers.  Be wary of similar but different cryptocurrencies like Bitcoin Cash.


Title: What is Bitcoin? - Start with Bitcoin
URL: https://startwithbitcoin.org/what-is-bitcoin
ID: https://startwithbitcoin.org/what-is-bitcoin
Published Date: None
Author: 
Image: https://startwit

In [5]:
def get_daily_closing_prices(ticker) -> pd.DataFrame:
    api_key = "OLEHY2C4W22JPZ9P"
    url = f"https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={ticker}&market=USD&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    price_data = data["Time Series (Digital Currency Daily)"]
    daily_close_prices = {
        date: prices["4. close"] for (date, prices) in price_data.items()
    }

    df = pd.DataFrame.from_dict(daily_close_prices, orient="index", columns=["price"])
    df.index = pd.to_datetime(df.index)
    df["price"] = pd.to_numeric(df["price"])

    return df

def format_prices(price_df: pd.DataFrame) -> str:
    """Format the daily closing prices."""
    text_output = [
        f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}"
        for date, row in price_df.head(10).iterrows()
    ]
    return "\n".join(text_output)

# Fetch and format Bitcoin price data
price_df = get_daily_closing_prices("BTC")
formatted_prices = format_prices(price_df)
print("\nBitcoin Daily Closing Prices:\n", formatted_prices)


Bitcoin Daily Closing Prices:
 2025-01-30 - 104454.86
2025-01-29 - 103747.25
2025-01-28 - 101290.00
2025-01-27 - 102062.42
2025-01-26 - 102563.00
2025-01-25 - 104733.56
2025-01-24 - 104850.27
2025-01-23 - 103926.36
2025-01-22 - 103667.11
2025-01-21 - 106159.26


#Tools

In [6]:
from crewai_tools import tool
from langchain.agents import load_tools

In [7]:
human_tools = load_tools(["human"])

In [8]:
@tool("search_tool")
def search_tool(query: str) -> str:
    """Search for the latest news and provide a summary about a given query using Exa."""
    # Perform the search and fetch the results
    result = exa.search_and_contents(query, summary=True)

    # Ensure results exist before processing
    if result.results:
        news_list = []
        for item in result.results:
            # Extracting attributes directly from the Result object
            news_item = {
                "title": item.title if hasattr(item, "title") else "No Title",
                "url": item.url if hasattr(item, "url") else "#",  # URL and ID are the same
                "id": item.id if hasattr(item, "id") else "#",  # ID is same as URL
                "score": item.score if hasattr(item, "score") else "No Score",
                "published_date": item.published_date if hasattr(item, "published_date") else "Unknown Date",
                "author": item.author if hasattr(item, "author") else "Unknown Author",
                "image": item.image if hasattr(item, "image") else "No Image",
                "favicon": item.favicon if hasattr(item, "favicon") else "No Favicon",
                "summary": item.summary if hasattr(item, "summary") else "No Summary",
                "highlights": item.highlights if hasattr(item, "highlights") else "No Highlights",
                "highlight_scores": item.highlight_scores if hasattr(item, "highlight_scores") else "No Highlight Scores",
            }
            news_list.append(news_item)

        # Format the news items into a readable string
        output = []
        for news_item in news_list:
            output.append(f"Title: {news_item['title']}\nURL: {news_item['url']}\nSummary: {news_item['summary']}\n")

        return "\n".join(output)
    else:
        return "No results found."

In [9]:
@tool("price tool")
def cryptocurrency_price_tool(ticker_symbol: str) -> str:
    """Get daily closing price for a given cryptocurrency ticker symbol for the previous 60 days"""
    price_df = get_daily_closing_prices(ticker_symbol)
    text_output = []
    for date, row in price_df.head(60).iterrows():
        text_output.append(f"{date.strftime('%Y-%m-%d')} - {row['price']:.2f}")
    return "\n".join(text_output)


In [10]:
@tool("search tool")
def cryptocurrency_news_tool(ticker_symbol: str) -> str:
    """Get news for a given cryptocurrency ticker symbol"""
    return search_tool.run(ticker_symbol + " cryptocurrency")

#Llama 3 with Groq

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [12]:
# Llama 3 with Groq for inference
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

system_message = "You are an experienced Machine Learning & AI Engineer."
human_message = "How to increase inference speed for a 7B LLM?"
prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", human_message)])

chain = prompt | llm
response = chain.invoke({"text": human_message})

print("\nLLM Response:\n", response.content)


LLM Response:
 Increasing inference speed for a 7B Large Language Model (LLM) is crucial for many applications, such as real-time language translation, chatbots, and text summarization. Here are some strategies to optimize inference speed for a 7B LLM:

1. **Model Pruning**: Remove redundant or unnecessary weights and connections in the model to reduce its size and computational requirements. This can be done using techniques like magnitude-based pruning, L1 regularization, or iterative pruning.
2. **Knowledge Distillation**: Train a smaller, simpler model (the student) to mimic the behavior of the large 7B LLM (the teacher). The student model can then be used for inference, which is typically faster than the teacher model.
3. **Quantization**: Represent the model's weights and activations using fewer bits (e.g., int8 instead of float32). This reduces memory usage and can lead to faster inference times. Techniques like quantization-aware training can help maintain accuracy.
4. **Tenso

#Agents

In [13]:
from crewai import Agent, Crew, Process, Task

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `CrewAgentExecutor` to V2.
  warn(


In [14]:
customer_communicator = Agent(
    role="Senior cryptocurrency customer communicator",
    goal="Find which cryptocurrency the customer is interested in",
    backstory="""You're highly experienced in communicating about cryptocurrencies
    and blockchain technology with customers and their research needs""",
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=human_tools,
)


In [15]:
news_analyst = Agent(
    role="Cryptocurrency News Analyst",
    goal="""Get news for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency news.
    You have a complete understanding of macroeconomic factors, but you specialize
    into analyzing news.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_news_tool],
)

In [16]:
price_analyst = Agent(
    role="Cryptocurrency Price Analyst",
    goal="""Get historical prices for a given cryptocurrency. Write 1 paragraph analysis of
    the market and make prediction - up, down or neutral.""",
    backstory="""You're an expert analyst of trends based on cryptocurrency
    historical prices. You have a complete understanding of macroeconomic factors,
    but you specialize into technical analys based on historical prices.
    """,
    verbose=True,
    allow_delegation=False,
    llm=llm,
    max_iter=5,
    memory=True,
    tools=[cryptocurrency_price_tool],
)

In [17]:
writer = Agent(
    role="Cryptocurrency Report Writer",
    goal="""Write 1 paragraph report of the cryptocurrency market.""",
    backstory="""
    You're widely accepted as the best cryptocurrency analyst that
    understands the market and have tracked every asset for more than 10 years. Your trends
    analysis are always extremely accurate.

    You're also master level analyst in the traditional markets and have deep understanding
    of human psychology. You understand macro factors and combine those multiple
    theories - e.g. cycle theory. You're able to hold multiple opininons when analysing anything.

    You understand news and historical prices, but you look at those with a
    healthy dose of skepticism. You also consider the source of news articles.

    Your most well developed talent is providing clear and concise summarization
    that explains very complex market topics in simple to understand terms.

    Some of your writing techniques include:

    - Creating a bullet list (executive summary) of the most importannt points
    - Distill complex analyses to their most important parts

    You writing transforms even dry and most technical texts into
    a pleasant and interesting read.""",
    llm=llm,
    verbose=True,
    max_iter=5,
    memory=True,
    allow_delegation=False,
)


#Tasks

In [18]:
get_cryptocurrency = Task(
    description=f"Ask which cryptocurrency the customer is interested in.",
    expected_output="""Cryptocurrency symbol that the human wants you to research e.g. BTC.""",
    agent=customer_communicator,
)

In [19]:
from datetime import datetime

get_news_analysis = Task(
    description=f"""
    Use the search tool to get news for the cryptocurrency

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph report for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=news_analyst,
    context=[get_cryptocurrency],
)

In [20]:
get_price_analysis = Task(
    description=f"""
    Use the price tool to get historical prices

    The current date is {datetime.now()}.

    Compose the results into a helpful report""",
    expected_output="""Create 1 paragraph summary for the cryptocurrency,
    along with a prediction for the future trend
    """,
    agent=price_analyst,
    context=[get_cryptocurrency],
)

In [21]:
write_report = Task(
    description=f"""Use the reports from the news analyst and the price analyst to
    create a report that summarizes the cryptocurrency""",
    expected_output="""1 paragraph report that summarizes the market and
    predicts the future prices (trend) for the cryptocurrency""",
    agent=writer,
    context=[get_news_analysis, get_price_analysis],
)

#Crew

In [22]:
crew = Crew(
    agents=[customer_communicator, price_analyst, news_analyst, writer],
    tasks=[get_cryptocurrency, get_news_analysis, get_price_analysis, write_report],
    verbose=2,
    process=Process.sequential,
    full_output=True,
    share_crew=False,
    manager_llm=llm,
    max_iter=15,
)

In [23]:
results = crew.kickoff()

 [DEBUG]: == Working Agent: Senior cryptocurrency customer communicator
 [INFO]: == Starting Task: Ask which cryptocurrency the customer is interested in.


> Entering new CrewAgentExecutor chain...
Thought: I need to ask the customer about their interest in cryptocurrencies to find out which one they're interested in.

Action: human
Action Input: {"question": "Hello! I'm here to help you with your cryptocurrency research. Are you interested in a specific cryptocurrency?"}

Hello! I'm here to help you with your cryptocurrency research. Are you interested in a specific cryptocurrency?
Dogecoin
 

Dogecoin

Thought: The customer mentioned Dogecoin, but I need to confirm if that's the one they're interested in researching.

Action: human
Action Input: {"question": "Great, you mentioned Dogecoin! Is that the cryptocurrency you'd like to learn more about?"}

Great, you mentioned Dogecoin! Is that the cryptocurrency you'd like to learn more about?
Yes
 

Yes

Thought: I now know the final an